# Groth16

## Introduction
Groth16 is a zk-SNARKs algorithm that allow a prover to prove that a statement is true, without revealing any other information about the input.

In this jupyter notebook, we are going to implement Groth16 to prove that our statement to a problem is true.

The main resource we need to implement it is the [Groth16 paper](https://eprint.iacr.org/2016/260.pdf).

## Context and Problem

In the following sections, we will implement and use Groth16 to prove a statement without leaking any other informations.

The problem that we are going to prove is: "I know $\{x,y,z\}$ such that $x^3+3x^2 y + 2z^2y^3 = 169695$".

## Problem's solution
The prover needs to know the values ${x,y,z}$.
In our example, those values are:
$$x = 3$$
$$y = 12$$
$$z = 7$$
Those values will remain private, they will never be sent to the verifier.

## Initialize Python setup
In our implementation, we will be working on the BN128 elliptic curve.
As such, we will use a finite field which is the number of point of this curve (the order of the curve).
We will need to adapt the our different computing to this.

In Python, we will work with the `galois` library for the finite field.
For the EC implementation, we will use `py_ecc`. `numpy` will be used to manipulate matrices.

In [1]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, pairing
import galois

print("Initializing a large field...")
GF = galois.GF(curve_order)
print("Field initialized")

Initializing a large field...
Field initialized


## Arithmetic Circuit
Our equation $out = x^3+3x^2 y + 2z^2y^3$ can be broken down into an arithmetic circuit.
The following scheme shows one of the multiple ways to represent our equation as an arithmetic circuit.

![Equation as an arithmetic circuit](./images/arithmetic-circuit.png)

From this arithmetic circuit, we can define a set of constraints which has the form $O = L * R$. *Note: in the following set of constraints, $v1, ..., v7$ represent intermediate values.*
$$v_1 = x * x$$<!-- x^2 -->
$$v_2 = v_1 * x$$<!-- x^3 -->
$$v_3 = 3v_1 * y$$<!-- 3x^2 y-->
$$v_4 = y * y$$<!-- y^2 -->
$$v_5 = v_4 * y$$<!-- y^3 -->
$$v_6 = 2z * z$$<!-- 2z^2-->
$$out = v_2 + v_3 + v_5 * v_6$$ <!-- x^3 + 3x^2 y + 2z^2 y^3-->

## Rank 1 Constraint System

Now that we have our arithmetic circuit, we can turn it into a Rank 1 Constraint System.
We need to adapt our set of constraints to easily match the required R1CS format:
$$v_1 = x * x$$<!-- x^2 -->
$$v_2 = v_1 * x$$<!-- x^3 -->
$$v_3 = 3v_1 * y$$<!-- 3x^2 y-->
$$v_4 = y * y$$<!-- y^2 -->
$$v_5 = v_4 * y$$<!-- y^3 -->
$$v_6 = 2z * z$$<!-- 2z^2-->
$$out - v_2 - v_3 = v_5 * v_6$$ <!-- x^3 + 3x^2 y + 2z^2 y^3-->

Now that we have a proper $O = L * R$ form, we can create 4 matrices.
The first matrix is named $s$. It is known as the *witness* matrix.
The value of $s$ elements are known by the prover.
$$s = \begin{bmatrix} 1 & out & x & y & z & v_1 & v_2 & v_3 & v_4 & v_5 & v_6 \end{bmatrix}$$
$$s = \begin{bmatrix} 1 & 169695 & 3 & 12 & 7 & 9 & 27 & 324 & 144 & 1728 & 98 \end{bmatrix}$$

Then, we can produce 3 matrices that will represent the constraints.
It sort of represents the way the $s$ elements are interconnected.
Those 3 matrices will be named $L$, $R$ and $O$.
$$L = \begin{bmatrix}
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 3 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 2 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
\end{bmatrix}$$
$$R = \begin{bmatrix}
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}$$
$$O = \begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 1 & 0 & 0 & 0 & 0 & -1 & -1 & 0 & 0 & 0 \\
\end{bmatrix}$$

The final R1CS representation is $Ls \odot Rs = Os$.
Each column of the matrices $L,R,O$ corresponds to an element of $s$.
Each row of the matrices $L,R,O$ corresponds to one of the constraints of the arithmetic circuit.

For example, let's take the 3rd row of $L, R, O$.
In $L$, the value $3$ is set at the 6th column.
The 6th column of $s$ is the $v_1$ variable.
So, the 3rd row of $L$ encodes $3 v_1$.
The same logic applies to $R$, its 3rd row encodes $y$.
And for $O$, the 3rd row encodes $v_3$.
If we put it all together, the $Os = Ls \odot Rs$ gives $v_3 = v_1 * y$ which correspond to our 3rd constraints defined previously.

Let's verify that the computing of $Os = Ls \odot Rs$ is correct.
We will use Python and numpy for this.
As we are working on a finite field, negative number will be modified.
For example, in $F_p$, $-1 (mod\ p)= p-1$.

In [2]:
import numpy as np

# define the matrices in our Galois Field

L = GF(np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]))
R = GF(np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]))


O = GF(np.array([
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 0, 0, curve_order-1, curve_order-1, 0, 0, 0]]))

# witness vector `s`
s = GF(np.array([1, 169695, 3, 12, 7, 9, 27, 324, 144, 1728, 98]))

# Calculate the result of the mutliplication of matrix, and check if it is equal to O*s
result = np.matmul(O, s) == np.matmul(L, s) * np.matmul(R, s)

# check that every element-wise equality is true
print("Os = Ls * Rs:", result.all())

Os = Ls * Rs: True


## QAP

We have a working R1CS corresponding to our arithmetic circuit.
The next step is to transform this R1CS into a Quadratic Arithmetic Program (QAP).

This transformation consists of turning our constraint matrices into polynomials.
The main goal of this transformation is **succinctness**.
Thanks to the Schwartz-Zippel Lemma, we know that evaluating two polynomials at a random point $x$ allows to nearly be certain that they are the same polynomial if they return the same value $y$.
In summary, if $P_1(\tau) == P_2(\tau)$, then $P1 == P2$, with $\tau$ being a random number.
This evaluation is way more succinct than matrices evaluation.

To achieve succinctness, we will derivate a QAP from our R1CS.
Our goal is to get $u * v = w$ if $Ls \odot Rs = Os$.
The transformation is made possible thanks to a ring homomorphism between matrices and polynomials.

The created polynomials must encode the same informations as our R1CS matrices, but uniquely at the points at which it will be evaluated.
As we have 7 constraints, we will evaluate our polynomials at $x=[1,2,3,4,5,6,7]$.

### R1CS to QAP
For each column of each matrix, we will interpolate a polynomial.
The interpolation will be done using Lagrange interpolation.
The interpolated polynomial will encode all the values of the column.

For example, the third column of $L$ is:
$$\begin{bmatrix} 1 \\ 1 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \end{bmatrix}$$

So, the interpolated polynomial $P_{L_3}$ must return the following values:
$$P_{L_3}(1) = 1$$
$$P_{L_3}(2) = 1$$
$$P_{L_3}(3) = 0$$
$$P_{L_3}(4) = 0$$
$$P_{L_3}(5) = 0$$
$$P_{L_3}(6) = 0$$
$$P_{L_3}(7) = 0$$

The following Python code shows the exact previous example (3rd column of matrix $L$).

In [3]:
x = GF(np.array([1,2,3,4,5,6,7]))
L3_column = GF(np.array([1,1,0,0,0,0,0]))

L3_poly = galois.lagrange_poly(x, L3_column)

print("The resulting polynomial is:\n", L3_poly, sep='')

# Checking each column element
print("L3_poly(1) == 1:", L3_poly(1) == 1)
print("L3_poly(2) == 1:", L3_poly(2) == 1)
print("L3_poly(3) == 0:", L3_poly(3) == 0)
print("L3_poly(4) == 0:", L3_poly(4) == 0)
print("L3_poly(5) == 0:", L3_poly(5) == 0)
print("L3_poly(6) == 0:", L3_poly(6) == 0)
print("L3_poly(7) == 0:", L3_poly(7) == 0)

The resulting polynomial is:
152001686609994966821155595453175521448252530558444682942348640184554225664x^6 + 9211302208565694989362029084462436599764103351841747786306327595183986075239x^5 + 18392204079809390985359827049834238095238556197571806636024185462331061305343x^4 + 10488116376089652710659736086269110979929424608532683123022056172734241570826x^3 + 14288158541339526881188625972598499016135737872493800196580772177348097212391x^2 + 13132945723103565133347843447154365053129018640249620606218922511945485097403x + 21888242871839275222246405745257275088548364400416034343698204186575808495603
L3_poly(1) == 1: True
L3_poly(2) == 1: True
L3_poly(3) == 0: True
L3_poly(4) == 0: True
L3_poly(5) == 0: True
L3_poly(6) == 0: True
L3_poly(7) == 0: True


Now that we achieved it for one column,
we can automate the process to get the interpolated polynomials for all the columns of each matrix.

The following Python code does it!

In [4]:
def interpolate_column(col, nb):
    xs = GF(np.arange(1,nb+1))
    return galois.lagrange_poly(xs, col)

def get_polys_of_matrix(matrix):
    polys = []
    nb_of_rows = len(matrix)
    nb_of_columns = len(matrix[0])
    # for each column
    for col_id in range(nb_of_columns):
        column = []
        for row in range(nb_of_rows):
            column.append(matrix[row][col_id])
        polys.append(interpolate_column(GF(np.array(column)), nb_of_rows))
    return np.array(polys)
    

## computes all interpolated polynomials for L, R, and O
U_polys = get_polys_of_matrix(L)
V_polys = get_polys_of_matrix(R)
W_polys = get_polys_of_matrix(O)

def get_polys_of_S_matrix(matrix):
    polys = []
    nb_of_columns = len(matrix)
    # for each column
    for col_id in range(nb_of_columns):
        polys.append(galois.Poly([matrix[0]], field=GF))
    return np.array(polys)
    
A_polys = get_polys_of_S_matrix(s)

# Summing all the polynamials of the collection into one polynomial
U = galois.Poly([0], field=GF)
V = galois.Poly([0], field=GF)
W = galois.Poly([0], field=GF)
for i in range(len(U_polys)):
    U += U_polys[i]
    V += V_polys[i]
    W += W_polys[i]

In [7]:
Ua = galois.Poly([0], field=GF)
for i in range(len(s)):
    Ua += U_polys[i] * s[i]

print(Ua(2))
print(Ua(3))
print(Ua(4))


Va = galois.Poly([0], field=GF)
for i in range(len(s)):
    Va += V_polys[i] * s[i]

Wa = galois.Poly([0], field=GF)
for i in range(len(s)):
    Wa += W_polys[i] * s[i]

3
27
12


### QAP balancing - $t(x)$ part
Our QAP $\sum\limits_{i=0}^{m} a_i u_i(x) \sum\limits_{i=0}^{m} a_i v_i(x) = \sum\limits_{i=0}^{m} a_i w_i(x)$ has an issue.
The left term will have a degree greater than the right term.
Even if this does not have any impact on the points at which we evaluate those polynomials, we will fix it by adding a *balancing term*.

The balancing term is a set of two polynomial $h(x)$ and $t(x)$ that will be added to the right term:
$\sum\limits_{i=0}^{m} a_i u_i(x) \sum\limits_{i=0}^{m} a_i v_i(x) = \sum\limits_{i=0}^{m} a_i w_i(x) + h(x)t(x)$.

We need this balancing term to be zero at the points we evaluate the polynomials. To do so, we use $t(x)$ as a sort of nullifier.
As we evaluate at $x=[1,...,7]$, we can easily find that $t(x)= (x-1)(x-2)...(x-7)$. Remember that as we are in a finite field, negative values are handled differently.

Then, we need to compute $h(x)$, this will be explained later.

To go further into the prover's steps, we need to initialize Groth16's trusted setup.

## Groth16
Now that we have almost prepared our QAP, we can get to the interesting part: Groth16.
To continue the prover steps, we need the trusted setup of Groth16.

### Trusted Setup
One of the most important part of the Groth16 algorithm is its trusted setup.
The values created during the process are highly sensitive.
If they are known by the prover, he will be able to forge correct proof without actually knowing the witness.

In our setup, we want two public inputs: $1$ and $out$.
The others variables of our circuits are private ones.

Our trusted setup will initialize the following values:
- powers of $\tau$ for $A$:
  - $\{\tau^i[G_1]_1\}^{n-1}_{i=0}$
- random shift for $A$:
  - $[\alpha G_1]_1$
- powers of $\tau$ for $B$:
  - $\{\tau^i[G_2]_2\}^{n-1}_{i=0}$
- random shift for $B$:
  - $[\beta G_2]_2$
- powers of $\tau$ for $public\ inputs$:
  - $\{\gamma^{-1} (\beta u_i (\tau^i) + \alpha v_i (\tau^i) + w_i(\tau^i)) [G_1]_1 \}^{l}_{i=0}$
- powers of $\tau$ for $private\ inputs$:
  - $\{\delta^{-1} (\beta u_i (\tau^i) + \alpha v_i (\tau^i) + w_i(\tau^i)) [G_1]_1 \}^{n-1}_{i=l+1}$
- powers of $\tau$ for $h(\tau)t(\tau)$:
  - $\{\delta^{-1} \tau^i t(\tau) [G_1]_1 \}^{n-2}_{i=0}$
- $\gamma$ and $\delta$:
  - $[\gamma G_2]_2, [\delta G_2]_2, [\delta G_2]_2$
- $[\beta G_1]_1$:
  - $[\beta G_1]_1$

*Note: $A$ and $B$ refer to the form $A * B = C$.*

The following code generates the trusted setup.

In [10]:
# tau value is random
tau = GF(20)
# alpha value is random
alpha = GF(888999666)
# beta value is random
beta = GF(111555777489)
# gamma value is random
gamma = GF(63708563)
inv_gamma = GF(curve_order-int(gamma))
# delta value is random
delta = GF(547345168978415)
inv_delta = GF(curve_order-int(delta))

# n degree of polynomials (degree 6)
n = len(L)-1
# public inputs
l = 2
# private inputs = total inputs - public inputs
m = len(L[0])
print(m)

# Computing t(x) - explained later
## T = (x-1)
T = galois.Poly([1, curve_order-1], field=GF)
## Then multiply by the other values: (x-2)(x-3)...(x-7)
for i in range(2, len(L)+1):
    print(i)
    T = T * galois.Poly([1, curve_order-i], field=GF)

# t(tau)
Ttau = T(tau)

# powers of tau for [A] (on G1)
tau_G1 = []
# powers of tau for [B] (on G2)
tau_G2 = []
for i in range(T.degree):
    print(i)
    local_tau = int(tau ** i)
    tau_G1.append(multiply(G1, local_tau))
    tau_G2.append(multiply(G2, local_tau))

print("tau G1 =", tau_G1)
print("tau G2 =", tau_G2)

# random shift for [A]
alpha_G1 = multiply(G1, int(alpha))
#random shift for [B]
beta_G2 = multiply(G2, int(beta))

# powers of tau for public inputs
"""public_G1 = []
for i in range(l):
    local_tau = int(tau ** i)
    #tmp = inv_gamma * (beta * U_polys[i](local_tau) + alpha * V_polys[i](local_tau) + W_polys[i](local_tau))
    tmp = inv_gamma * (beta * U_polys[i](local_tau) + alpha * V_polys[i](local_tau) + W_polys[i](local_tau))
    public_G1.append(multiply(G1, int(tmp)))

# powers of tau for private inputs
private_G1 = []
for i in range(l, m):
    local_tau = int(tau ** i)
    tmp = inv_delta * (beta * U_polys[i](local_tau) + alpha * V_polys[i](local_tau) + W_polys[i](local_tau))
    private_G1.append(multiply(G1, int(tmp)))"""

# powers of tau for h(tau)t(tau)
t_G1 = []
for i in range(T.degree-1):
    local_tau = int(tau ** i)
    tmp = (local_tau * T(tau))
    t_G1.append(multiply(G1, int(tmp)))

# powers of tau for C
C_tau = []
for i in range(T.degree-1):
    local_tau = int(tau ** i)
    num = beta * U_polys[i](local_tau) + alpha * V_polys[i](local_tau) + W_polys[i](local_tau)
    C_tau.append(multiply(G1, int(num)))

# Gamma and Delta EC points on G2
gamma_G2 = multiply(G2, int(gamma))
delta_G2 = multiply(G2, int(delta))
# delta_G1 and beta_G1
delta_G1 = multiply(G1, int(delta))
beta_G1 = multiply(G1, int(beta))

11
2
3
4
5
6
7
0
1
2
3
4
5
6
tau G1 = [(1, 2), (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992), (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714), (21603600070689675766438470661345954782419355034652174505468210225883925863279, 15787091953565760722773063158476721787069408761080596737736006929439659337677), (3791913980001525405070663195453841654293855276471519589821575313643995787424, 2219850731288481436925303713906758446890789653022769553096390029843417460412), (6779621809516151986945641406336169543842908834692441338288564144351856901573, 1474660080685150469507045477646463875368452658327288987469380125599003345221), (15843612621971795447506732376115451795680130889363554944659411839708446467847, 11883150635389841371602016045140019253999041521943292833653793077720618883158

# tau value is random
tau = GF(123456789)
# alpha value is random
alpha = GF(888999666)
# beta value is random
beta = GF(111555777489)
# gamma value is random
gamma = GF(63708563)
inv_gamma = GF(curve_order-int(gamma))
# delta value is random
delta = GF(547345168978415)
inv_delta = GF(curve_order-int(delta))

# n degree of polynomials (degree 6)
n = len(L)-1
# public inputs
l = 2
# private inputs = total inputs - public inputs
m = len(L[0])

# Computing t(x) - explained later
## T = (x-1)
T = galois.Poly([1, curve_order-1], field=GF)
## Then multiply by the other values: (x-2)(x-3)...(x-7)
for i in range(2, len(L)+1):
    T = T * galois.Poly([1, curve_order-i], field=GF)

# t(tau)
Ttau = T(tau)

# powers of tau for [A] (on G1)
tau_G1 = []
# powers of tau for [B] (on G2)
tau_G2 = []
for i in range(n+1):
    local_tau = int(tau ** i)
    tau_G1.append(multiply(G1, local_tau))
    tau_G2.append(multiply(G2, local_tau))

# random shift for [A]
alpha_G1 = multiply(G1, int(alpha))
#random shift for [B]
beta_G2 = multiply(G2, int(beta))

# powers of tau for public inputs
"""public_G1 = []
for i in range(l):
    local_tau = int(tau ** i)
    #tmp = inv_gamma * (beta * U_polys[i](local_tau) + alpha * V_polys[i](local_tau) + W_polys[i](local_tau))
    tmp = inv_gamma * (beta * U_polys[i](local_tau) + alpha * V_polys[i](local_tau) + W_polys[i](local_tau))
    public_G1.append(multiply(G1, int(tmp)))

# powers of tau for private inputs
private_G1 = []
for i in range(l, m):
    local_tau = int(tau ** i)
    tmp = inv_delta * (beta * U_polys[i](local_tau) + alpha * V_polys[i](local_tau) + W_polys[i](local_tau))
    private_G1.append(multiply(G1, int(tmp)))"""

"""# powers of tau for h(tau)t(tau)
t_G1 = []
for i in range(n):
    local_tau = int(tau ** i)
    tmp = inv_delta * (local_tau * T(tau))
    t_G1.append(multiply(G1, int(tmp)))"""

# powers of tau for C
C_tau = []
for i in range(n+1):
    local_tau = int(tau ** i)
    num = beta * U_polys[i](local_tau) + alpha * V_polys[i](local_tau) + W_polys[i](local_tau)
    C_tau.append(multiply(G1, int(num)))

# Gamma and Delta EC points on G2
gamma_G2 = multiply(G2, int(gamma))
delta_G2 = multiply(G2, int(delta))
# delta_G1 and beta_G1
delta_G1 = multiply(G1, int(delta))
beta_G1 = multiply(G1, int(beta))

In [11]:
# Printing the trusted setup resulting output (all the public values)
print("Values used for A term:\n", tau_G1)
print('---'*20)
print("Values used for B term:\n", tau_G2)
print('---'*20)
print("Random shift for A term:\n", alpha_G1)
print('---'*20)
print("Random shift for B term:\n", beta_G2)
print('---'*20)
print("Values used for public inputs:\n", public_G1)
print('---'*20)
print("Values used for private inputs:\n", private_G1)
print('---'*20)
print("Values used for h(x)t(x):\n", private_G1)
print('---'*20)
print("Values used for gamma G2 point:\n", gamma_G2)
print('---'*20)
print("Values used for delta G2 point:\n", delta_G2)
print('---'*20)

Values used for A term:
 [(1, 2), (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992), (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714), (21603600070689675766438470661345954782419355034652174505468210225883925863279, 15787091953565760722773063158476721787069408761080596737736006929439659337677), (3791913980001525405070663195453841654293855276471519589821575313643995787424, 2219850731288481436925303713906758446890789653022769553096390029843417460412), (6779621809516151986945641406336169543842908834692441338288564144351856901573, 1474660080685150469507045477646463875368452658327288987469380125599003345221), (15843612621971795447506732376115451795680130889363554944659411839708446467847, 11883150635389841371602016045140019253999041521943292833653793077720618883158)]
----------

NameError: name 'public_G1' is not defined

### QAP balancing - $h(x)$ part
BLABLABLA

In [12]:
H = (Ua * Va - Wa) // T
print("t(x):", T)
print("h(x):", H)
print("rem=", (Ua * Va - Wa)%T)
print("coefs H:", H.coefficients())
print("Ua(tau) * Va(tau) - Wa(tau) == H(T(tau)):", Ua(tau) * Va(tau) - Wa(tau) == H(tau)*T(tau))

t(x): x^7 + 21888242871839275222246405745257275088548364400416034343698204186575808495589x^6 + 322x^5 + 21888242871839275222246405745257275088548364400416034343698204186575808493657x^4 + 6769x^3 + 21888242871839275222246405745257275088548364400416034343698204186575808482485x^2 + 13068x + 21888242871839275222246405745257275088548364400416034343698204186575808490577
h(x): 8315758938288474643174054034584144152564815525968244529304323523429987429036x^5 + 14422933370133722418436717267233647978486521782922287815456321304978401959157x^4 + 17050029187043135428329023141982698240850486352740740085643246969501447492822x^3 + 11769237258067560289350709163280291999869113020256107725754484628423059435872x^2 + 4565623993475548820084776902095215412567345176207150126645011989010060425174x + 20702629716281314481041392100722506021251994662060165816747884793136285535344
rem= 0
coefs H: [ 8315758938288474643174054034584144152564815525968244529304323523429987429036
 144229333701337224184367172672336479784865

### Adding the witness to the trusted setup polynomials

Once the first QAP has been used to generate the variables during the trusted setup, we can use those variables.
This witness plug could have been done before to verify our QAP polynomials.

Here are the formulas that the prover needs to compute:

$$[A]_1 = [\alpha]_1 + \sum\limits_{i=0}^{m} a_i [u_i(\tau)]_1 + r[\delta]_1$$

$$[B]_2 = [\beta]_2 + \sum\limits_{i=0}^{m} a_i [v_i(\tau)]_2 + s[\delta]_2$$

$$[B]_1 = [\beta]_1 + \sum\limits_{i=0}^{m} a_i [v_i(\tau)]_1 + s[\delta]_1$$

$$[C]_1 = \sum\limits_{i=0}^{m} a_i [\beta u_i(\tau) + \alpha v_i (\tau) + w_i(\tau)]_1 + [h(\tau)t(\tau)]_1 + s[A]_1 + r[B]_1 - rs[\delta]_1$$


In [13]:
def poly_hiding(poly, points, group):
    coefs = poly.coefficients()[::-1]
    assert len(coefs) == len(points), "Length mismatch"
    if group == 1:
        result = multiply(G1, 0)
    if group == 2:
        result = multiply(G2, 0)
    for i in range(len(coefs)):
        result = add(result, multiply(points[i], int(coefs[i])))
    return result

# [A]_1 generation
A_1 = poly_hiding(Ua, tau_G1, 1)
#A_1 = add(add(alpha_G1, tmp_A), multiply(delta_G1, int(R)))
#A_1 = add(alpha_G1, tmp_A)


# [B]_1 generation
B_1 = poly_hiding(Va, tau_G1, 1)
#B_1 = add(add(beta_G1, tmp_B), multiply(delta_G1, int(S)))
#B_1 = add(beta_G1, tmp_B)

# [B]_2 generation
B_2 = poly_hiding(Va, tau_G2, 2)
#B_2 = add(add(beta_G2, tmp_B), multiply(delta_G2, int(S)))
#B_2 = add(beta_G2, tmp_B)



## h(tau)t(tau)
HT_tau = poly_hiding(H, t_G1, 1)
tmp_Cpriv = poly_hiding(Wa, tau_G1, 1)
# assert len(s) == len(private_G1) + len(public_G1), "error"
#constant_part = add(add(multiply(A_1, int(S)), multiply(B_1, int(R))), multiply(delta_G1, int(R*S)))
## hiding priv points
#tmp_Cpriv = multiply(G1, 0)
#tmp_Cpriv = poly_hiding(Wa, C_tau,1)
#for i in range(2, len(s)):
#    tmp_Cpriv = add(tmp_Cpriv, multiply(private_G1[i-2], int(s[i])))
#for i in range(0, len(s)):
#    tmp_Cpriv = add(tmp_Cpriv, multiply(C_tau[i], int(s[i])))

# C_1 = alpha beta + delta^{-1} (sum(private inputs) + ht(x) + As +rB - rs delta
#C_1 = add(multiply(add(tmp_Cpriv, HT_tau), int(inv_delta)), constant_part)
C_1 = add(tmp_Cpriv, HT_tau)


print("\nProof verification:")
print("-"*10)
# e(A, B) == e(C, G2[1])
print("e(A, B) == e(C, G2[1]):",pairing(B_2, A_1) == pairing(G2, C_1))
print("Pairing check passed!")


Proof verification:
----------
e(A, B) == e(C, G2[1]): False
Pairing check passed!


# prover generates r and s, randoms to protect its inputs
R = GF(97842355664152)
S = GF(123445612456000445)

def poly_hiding(poly, points, group):
    coefs = poly.coefficients()
    assert len(coefs) == len(points), "Length mismatch"
    if group == 1:
        result = multiply(G1, 0)
    if group == 2:
        result = multiply(G2, 0)
    for i in range(len(coefs)):
        result = add(result, multiply(points[i], int(coefs[i])))
    return result

# [A]_1 generation
tmp_A = poly_hiding(Ua, tau_G1, 1)
#A_1 = add(add(alpha_G1, tmp_A), multiply(delta_G1, int(R)))
A_1 = add(alpha_G1, tmp_A)


# [B]_1 generation
tmp_B = poly_hiding(Va, tau_G1, 1)
#B_1 = add(add(beta_G1, tmp_B), multiply(delta_G1, int(S)))
B_1 = add(beta_G1, tmp_B)

# [B]_2 generation
tmp_B = poly_hiding(Va, tau_G2, 2)
#B_2 = add(add(beta_G2, tmp_B), multiply(delta_G2, int(S)))
B_2 = add(beta_G2, tmp_B)

## h(tau)t(tau)
HT_tau = poly_hiding(H, t_G1, 1)
assert len(s) == len(private_G1) + len(public_G1), "error"
#constant_part = add(add(multiply(A_1, int(S)), multiply(B_1, int(R))), multiply(delta_G1, int(R*S)))
## hiding priv points
#tmp_Cpriv = multiply(G1, 0)
tmp_Cpriv = poly_hiding(Wa, C_tau,1)
#for i in range(2, len(s)):
#    tmp_Cpriv = add(tmp_Cpriv, multiply(private_G1[i-2], int(s[i])))
#for i in range(0, len(s)):
#    tmp_Cpriv = add(tmp_Cpriv, multiply(C_tau[i], int(s[i])))

# C_1 = alpha beta + delta^{-1} (sum(private inputs) + ht(x) + As +rB - rs delta
#C_1 = add(multiply(add(tmp_Cpriv, HT_tau), int(inv_delta)), constant_part)
C_1 = add(tmp_Cpriv, HT_tau)

### Verifier steps



In [234]:
first = pairing(B_2, A_1)
second = pairing(beta_G2, alpha_G1) + pairing(G2, C_1)
print("first pairing == second pairing:", first == second)
print("First:", first)
print("Second:", second)

first pairing == second pairing: False
First: (983731774277570230104737829635417331974818914425264848661439543061644127987, 18277631016736721106110908792481622714502053619362760560476407591482334594820, 1581501987921264759668286212406943198041218649962658448182483910454722124554, 10625419883786678862887023310106454074917374298872668220951909589675697215640, 12032873804074978763688887458403275004275023360398369595251801642336707590693, 13422667100093579372081642152754695301910521457581535518502895996222284047576, 9802592052914985903846939240193926024996739425813034963796830504884325939697, 14645508004100005544625762962837114880000582701155106497130343237293041210858, 3804943694884243104915022527292712572086227059024751138048053803731014200047, 21630889487314970230130571243863527099800644795603380019340050979152162064839, 7450725925627747680403374232756616837648926988637630442926492598577838031051, 18221257118989982997191103783224530325804532446743800057121155964109959703427)
Second: (71

In [227]:
# calculate public inputs
tmp_Cpub = multiply(G1, 0)
for i in range(0, 2):
    tmp_Cpub = add(tmp_Cpub, multiply(public_G1[i], int(s[i])))

first = pairing(B_2, A_1)
second = pairing(beta_G2, alpha_G1) + pairing(delta_G2, C_1) + pairing(gamma_G2, tmp_Cpub)
print("first pairing == second pairing:", first == second)
print("First:", first)
print("Second:", second)

first pairing == second pairing: False
First: (983731774277570230104737829635417331974818914425264848661439543061644127987, 18277631016736721106110908792481622714502053619362760560476407591482334594820, 1581501987921264759668286212406943198041218649962658448182483910454722124554, 10625419883786678862887023310106454074917374298872668220951909589675697215640, 12032873804074978763688887458403275004275023360398369595251801642336707590693, 13422667100093579372081642152754695301910521457581535518502895996222284047576, 9802592052914985903846939240193926024996739425813034963796830504884325939697, 14645508004100005544625762962837114880000582701155106497130343237293041210858, 3804943694884243104915022527292712572086227059024751138048053803731014200047, 21630889487314970230130571243863527099800644795603380019340050979152162064839, 7450725925627747680403374232756616837648926988637630442926492598577838031051, 18221257118989982997191103783224530325804532446743800057121155964109959703427)
Second: (20